In [193]:
import pandas as pd
import pathlib

In [194]:
file = pathlib.Path.cwd() / "Enfield Saturn Routes AM test.xlsx"

In [195]:
strategic_data = pd.read_excel(file, skiprows=1, header=None, dtpe=object)
qgis_table = pd.read_excel(file, sheet_name=1, header=0, usecols=["AssANode","AssBNode", "ID"], index_col=None)

In [196]:
strategic_data=strategic_data[strategic_data.iloc[:,0] == "route"]

In [197]:
nodes = strategic_data.to_string(header=False, index=False).split()

In [198]:
nodes=list(filter(lambda x: "NaN" not in x, nodes))

In [199]:
nodes_grouped,count = [],-1 # todo change from -1
for i in range(len(nodes)):
    if nodes[i] == "route":
        nodes_grouped.append([])
        count+=1
        continue
    if "+" in nodes[i]:
        nodes_grouped[count].append(float(nodes[i].replace("+","")))
    else:
        nodes_grouped[count].append(float(nodes[i]))

In [200]:
for count, node_group in enumerate(nodes_grouped):
    if len(str(node_group[-1])) < len(str(node_group[0])): # Think of a more robust way, perhaps average the lengths of all but the last
        node_group[-1] = node_group[-1]*100
    nodes_grouped[count] = list(map(int,node_group))

In [201]:
nodes_joined=[]
for i in range(len(nodes_grouped)):
    nodes_joined.append([])
    for j in range(len(nodes_grouped[i])-1):
        nodes_joined[i].append(f"{nodes_grouped[i][j]}>{nodes_grouped[i][j+1]}")

In [202]:
# Extract the ID for each node to node combination, representing a link.

In [203]:
links=[]
for i in range(len(nodes_joined)):
    links.append([])
    for link in nodes_joined[i]:
        link_index = (qgis_table.index[qgis_table["AssBNode"] == link].tolist()[0])
        links[i].append(qgis_table.at[link_index, "ID"])

In [204]:
links

[[204, 201, 172, 210, 205, 176, 194, 191, 181],
 [204, 202, 170],
 [204, 202, 169],
 [204, 201, 172, 210, 205, 176, 193, 206, 177, 199],
 [204, 201, 172, 210, 205, 176, 194, 191, 180, 195],
 [204, 201, 172, 210, 205, 176, 194, 191, 183],
 [208, 182, 189, 193, 206, 178, 167, 214, 209, 173, 203],
 [208, 182, 189, 193, 206, 178, 167, 214, 209, 172, 211, 219],
 [208, 182, 189, 193, 206, 178, 166, 217],
 [208, 182, 189, 193, 206, 178, 167, 214, 209, 173, 202, 170],
 [208, 182, 189, 193, 206, 178, 167, 214, 209, 173, 202, 169],
 [208, 182, 189, 193, 206, 177, 199],
 [208, 183],
 [208, 182, 190],
 [215, 172, 211, 220, 209, 173, 203],
 [215, 172, 210, 205, 176, 194, 191, 181],
 [215, 172, 211, 219],
 [215, 172, 210, 205, 176, 193, 206, 178, 166, 217],
 [215, 172, 211, 220, 209, 173, 202, 170],
 [215, 172, 211, 220, 209, 173, 202, 169],
 [215, 172, 210, 205, 176, 193, 206, 177, 199],
 [215, 172, 210, 205, 176, 194, 191, 180, 195],
 [215, 172, 210, 205, 176, 194, 191, 183],
 [215, 172, 210, 205,

routes=[]
for route in links:
    print(" ".join(map(str,route)))